In [1]:
from flask import Flask, render_template, Response
import cv2
import mediapipe as mp
import pandas as pd
import datetime
import time
import pymysql
import os

In [2]:
app = Flask(__name__)

con = pymysql.connect(host='project-db-campus.smhrd.com', user='sc_24K_bigdata25_p3_4', password='smhrd4',
                      port=3312, db='sc_24K_bigdata25_p3_4', charset='utf8', # 한글처리 (charset = 'utf8')
                      autocommit=True # 결과 DB 반영 (Insert or update)
                     )
cur = con.cursor()

# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 랜드마크 선택
selected_landmarks = [0] + list(range(11, 17)) + list(range(23, 29))

# 컬럼 이름 정의
columns = ['frame'] + [f'landmark{idx}_{axis}' for idx in selected_landmarks for axis in ['x', 'y']] + \
          ['neck_x', 'neck_y', 'hip_x', 'hip_y']

def generate_frames(id, ps_idx):
    print('generate_frames 실행' ,id , ps_idx)
    user_id = id
    select_yoga = ps_idx
    data = []
    cap = cv2.VideoCapture(0)  # 카메라 열기
    frame_idx = 0
    start_time = time.time()
    duration = 30  # 30초 동안 실행

    while cap.isOpened() and time.time() - start_time < duration:
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        frame_data = [frame_idx]

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            for idx in selected_landmarks:
                landmark = landmarks[idx]
                frame_data.extend([landmark.x, landmark.y])

            # 목과 엉덩이 좌표 계산
            neck_x = (landmarks[11].x + landmarks[12].x) / 2
            neck_y = (landmarks[11].y + landmarks[12].y) / 2
            hip_x = (landmarks[23].x + landmarks[24].x) / 2
            hip_y = (landmarks[23].y + landmarks[24].y) / 2
            frame_data.extend([neck_x, neck_y, hip_x, hip_y])

        else:
            frame_data.extend([None, None] * len(selected_landmarks))
            frame_data.extend([None, None, None, None])

        data.append(frame_data)

        # 시각화
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            h, w, _ = frame.shape

            # 점 그리기
            for idx in selected_landmarks:
                x, y = int(landmarks[idx].x * w), int(landmarks[idx].y * h)
                cv2.circle(frame, (x, y), 5, (255, 0, 0), -1)  # 파란색 점

            # 선 그리기
            for connection in mp_pose.POSE_CONNECTIONS:
                start_idx, end_idx = connection
                if start_idx in selected_landmarks and end_idx in selected_landmarks:
                    start_x, start_y = int(landmarks[start_idx].x * w), int(landmarks[start_idx].y * h)
                    end_x, end_y = int(landmarks[end_idx].x * w), int(landmarks[end_idx].y * h)
                    cv2.line(frame, (start_x, start_y), (end_x, end_y), (255, 255, 255), 2)  # 하얀색 선

        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

    cap.release()
    pose.close()
    
    # CSV 저장
    suffix = datetime.datetime.now().strftime('%Y-%m-%d_%H%M%S')
    output_csv_path = f'./BPE/training/{user_id}_{suffix}.csv'
    filepath = f'{user_id}_{suffix}'
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(output_csv_path, index=False)
    print(f"Joint 데이터를 {output_csv_path}에 저장했습니다.")

    #BPE 로 이동
    os.chdir('C:\\Users\\USER\\yogafire\\BPE')
    
    #BPE 안의 모델 파일 실행 / training 안에 방금 저장된 유저의 파일과 , rightyogajoint에 미리 저장해둔 요가 자세의 올바른 자세 joint 값과 비교 
    output = os.popen(f"python inference_pair.py -v1 training/{filepath}.csv -v2 rightyogajoint/{select_yoga}.csv").read()
    print(output)
    # 실행된 결과에서 score만 가져오기
    list1 = output.split('\n')
    score_str = list1[-2]
    # score 점수 조정
    model_score = ((round(sigmoid((float(score_str.split(' ')[-1])))*10)*10))
    print(model_score)

    # activity_at, activity_score 값을 적절하게 정의합니다. 예시로 현재 시간과 랜덤 점수를 사용
    activity_at = datetime.datetime.now().strftime('%Y-%m-%d')  # 활동 시간
    activity_score = model_score  # 예시 점수
    insert_user_data(id, flow_order_idx[0], activity_at, activity_score)
    


def recent_order(id, flow_idx) :
    # activity_at -> datetime으로 바꿀수잇는지 물어보기
    sql = 'select flow_ord from tmp where (id = %s) and (flow_idx = %s) limit 1'
    cur.execute(sql, (id,flow_idx))
    flow_ord = cur.fetchone()
    if recent_order :
        print('가장 최근의 flow_ord :', flow_ord[0])
        return flow_ord[0]
    else :
        print('id에 대한 최근 flow_ord가 없습니다.')
        return 1

def select_ps_data(flow_idx, flow_ord) :
    # flow_ord와 flow_idx에 해당하는 ps_idx를 찾는 쿼리
    sql = 'select ps_idx from flow_order where flow_idx = %s and flow_ord = %s'
    cur.execute(sql, (flow_idx, flow_ord))
    ps_idx = cur.fetchone()  # 쿼리 결과에서 첫 번째 값만 가져옵니다.  
    print(ps_idx)
    return ps_idx

def select_order_data(flow_idx, flow_ord) :
    print('select order data')
    sql = 'select flow_order_idx from flow_order where flow_idx = %s and flow_ord = %s'
    cur.execute(sql, (flow_idx, flow_ord))
    flow_order_idx = cur.fetchone()  # 쿼리 결과에서 첫 번째 값만 가져옵니다.
    print(flow_order_idx)
    return flow_order_idx

def select_flow_data(flow_idx) :
    sql = 'select * from flow_info where flow_idx = %s'
    cur.execute(sql, (flow_idx,))
    flow_info = cur.fetchall()
    print('select flow data 실행')
    return flow_info

def select_pose_data(ps_idx) :
    sql = 'select* from pose_info where ps_idx = %s'
    cur.execute(sql, (ps_idx,))
    ps_info = cur.fetchall()
    print('select pose data 실행')
    return ps_info

def insert_user_data(id, flow_order_idx, activity_at, activity_score) :
    print('insert user data')
    sql = 'insert into user_activity (id, flow_order_idx, activity_at, activity_score) values (%s, %s, %s, %s)'
    cur.execute(sql, (id, flow_order_idx, activity_at, activity_score))
    con.commit()


In [ ]:

@app.route('/video_feed/<id>/<ps_idx>')
def video_feed(id,ps_idx):
    return Response(generate_frames(id,ps_idx), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/boot/programs/<id>/<flow_idx>')
def index(id, flow_idx):
    
    flow_ord = recent_order(id,flow_idx)

    ps_idx = select_ps_data(flow_idx, flow_ord)

    flow_order_idx = select_order_data(flow_idx, flow_ord)


    flow_info = select_flow_data(flow_idx)
    pose_info = select_pose_data(ps_idx)
    print(pose_info)

    return render_template('programs.html', id=id, flow_idx=flow_idx, flow_title = flow_info[0][2], 
                           ps_idx = pose_info[0][0],
                           ps_name = pose_info[0][1],  # '웃타나사나'
                           ps_time = pose_info[0][2],  # 30
                           ps_exp = pose_info[0][3],   # 자세 설명
                           ps_img = pose_info[0][4],   # '1-1.jpg'
                           ps_cal = pose_info[0][5],   # 100
                           ps_link = pose_info[0][6],
                          flow_ord_index = flow_order_idx)  # 링크)
      
if __name__ == "__main__":
    app.run(host='localhost', port=8097)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8097
Press CTRL+C to quit
